In [2]:
pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.7 MB/s eta 0:00:00


In [3]:
from rdflib import Graph
from rdflib.namespace import FOAF , XSD
from rdflib.plugins.sparql import prepareQuery
import requests

In [4]:
# Define the URI of the RDF file in your GitHub repository
github_raw_uri = "https://raw.githubusercontent.com/mtbstn24/OntoML/main/OntoML.rdf"

# Fetch the RDF file from the GitHub repository
response = requests.get(github_raw_uri)

if response.status_code == 200:
  # Create a Graph
  g = Graph()
  # Parse in an RDF file hosted on the Internet
  g.parse(data=response.text, format="application/rdf+xml")

  # Loop through each triple in the graph (subj, pred, obj)
  for subj, pred, obj in g:
      # Check if there is at least one triple in the Graph
      if (subj, pred, obj) not in g:
        raise Exception("It better be!")

  # Print the number of "triples" in the Graph
  print(f"Graph g has {len(g)} statements.")

  # Print out the entire Graph in the RDF Turtle format
  print(g.serialize(format="turtle"))

else:
  print("Failed to fetch the RDF file from GitHub.")

Graph g has 220 statements.
@prefix : <https://github.com/mtbstn24/OntoML> .
@prefix OntoML: <https://github.com/mtbstn24/OntoML#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

OntoML:affectsPlantPart a owl:ObjectProperty ;
    rdfs:domain OntoML:Disease ;
    rdfs:range OntoML:Part .

OntoML:bacterial_spot a owl:NamedIndividual,
        OntoML:Disease ;
    OntoML:affectsPlantPart OntoML:fruit,
        OntoML:leaf,
        OntoML:stem ;
    OntoML:hasDiseaseColour OntoML:brown,
        OntoML:yellow ;
    OntoML:hasPlantStage OntoML:any_stage ;
    OntoML:hasSymptom OntoML:fruit_halo,
        OntoML:spots .

OntoML:curling a owl:NamedIndividual,
        OntoML:Symptom .

OntoML:hasAffectedPlantPart a owl:ObjectProperty ;
    rdfs:domain OntoML:Symptom ;
    rdfs:range OntoML:Part .

OntoML:hasAffectedSeason a owl:ObjectProperty ;
    rdfs:domain OntoML:Disease ;
    rdfs:range OntoML:Season .

OntoML:hasC

In [5]:
from rdflib.plugins.sparql import prepareQuery

s = ["spots", "lesions"]
c = ["brown"]
p = ["fruit", "leaf"]
stages = []

# conditions check whether all specified symptoms, colours and parts are related to the disease (AND is checked, not OR)

symptoms = " ".join([f"""
  ?disease OntoML:hasSymptom OntoML:{symptom}.
""" for symptom in s])

colors = " ".join([f"""
  ?disease OntoML:hasDiseaseColour OntoML:{color}.
""" for color in c])

affectedParts = " ".join([f"""
  ?disease OntoML:affectsPlantPart OntoML:{part}.
""" for part in p])

plantStages = " ".join([f"""
  ?disease OntoML:hasPlantStage OntoML:{stage}.
""" for stage in stages])

sample_sparql_query = f"""
  PREFIX OntoML: <https://github.com/mtbstn24/OntoML#>
   SELECT DISTINCT (strafter(STR(?disease), "#") AS ?diseaseName)
WHERE {{
  ?disease rdf:type OntoML:Disease.
  {symptoms}
  {colors}
  {affectedParts}
  {plantStages}
}}
"""

query = prepareQuery(sample_sparql_query)
# results = g.query(query)

for r in g.query(query):
    disease = r.diseaseName
    print(f"Disease: {disease}, Symptom: {s}, SymptomColour: {c}, AffectsPart: {p}")

Disease: target_spot, Symptom: ['spots', 'lesions'], SymptomColour: ['brown'], AffectsPart: ['fruit', 'leaf']


In [6]:
def queryDisease(s, c, p, stages):
  diseases = []

  # conditions check whether all specified symptoms, colours and parts are related to the disease (AND is checked, not OR)
  symptoms = " ".join([f"""
    ?disease OntoML:hasSymptom OntoML:{symptom}.
  """ for symptom in s])

  colors = " ".join([f"""
    ?disease OntoML:hasDiseaseColour OntoML:{color}.
  """ for color in c])

  affectedParts = " ".join([f"""
    ?disease OntoML:affectsPlantPart OntoML:{part}.
  """ for part in p])

  plantStages = " ".join([f"""
    ?disease OntoML:hasPlantStage OntoML:{stage}.
  """ for stage in stages])

  sample_sparql_query = f"""
    PREFIX OntoML: <https://github.com/mtbstn24/OntoML#>
    SELECT DISTINCT (strafter(STR(?disease), "#") AS ?diseaseName)
  WHERE {{
    ?disease rdf:type OntoML:Disease.
    {symptoms}
    {colors}
    {affectedParts}
    {plantStages}
  }}
  """

  query = prepareQuery(sample_sparql_query)
  # results = g.query(query)

  for r in g.query(query):
    disease = r.diseaseName
    print(f"Disease: {disease}, Symptom: {s}, SymptomColour: {c}, AffectsPart: {p}")
    diseases.append(f"{disease}")

  return diseases

In [7]:
s = ['lesions', 'rings']
c = []
p = ['leaf']
stages = []
diseases = queryDisease(s,c,p,stages)

Disease: early_blight, Symptom: ['lesions', 'rings'], SymptomColour: [], AffectsPart: ['leaf']
Disease: target_spot, Symptom: ['lesions', 'rings'], SymptomColour: [], AffectsPart: ['leaf']


In [8]:
print(diseases)

['early_blight', 'target_spot']


In [9]:
#requires colours array to be ordered according to the symptom set
# s = ['lesions', 'rings', 'yellowing']
# c = ['brown', 'brown', 'yellow']
def queryDiseaseusingSymptomColor(s, c, p, stages):
  diseases = []

  # conditions check whether all specified symptoms, colours and parts are related to the disease (AND is checked, not OR)
  symptoms = " ".join([f"""
    ?disease OntoML:hasSymptom OntoML:{symptom}.
  """ for symptom in s])

  colors = " ".join([f"""
    ?symptom OntoML:hasSymptomColour OntoML:{color}.
  """ for color in c])

  affectedParts = " ".join([f"""
    ?disease OntoML:affectsPlantPart OntoML:{part}.
  """ for part in p])

  plantStages = " ".join([f"""
    ?disease OntoML:hasPlantStage OntoML:{stage}.
  """ for stage in stages])

  sample_sparql_query = f"""
    PREFIX OntoML: <https://github.com/mtbstn24/OntoML#>
    SELECT DISTINCT (strafter(STR(?disease), "#") AS ?diseaseName)
  WHERE {{
    ?disease rdf:type OntoML:Disease.
    {symptoms}
    {colors}
    {affectedParts}
    {plantStages}
  }}
  """

  query = prepareQuery(sample_sparql_query)
  # results = g.query(query)

  for r in g.query(query):
    disease = r.diseaseName
    print(f"Disease: {disease}, Symptom: {s}, SymptomColour: {c}, AffectsPart: {p}")
    diseases.append(f"{disease}")

  return diseases

In [10]:
s = ['lesions']
c = ['black']
p = ['leaf']
stages = []
diseases = queryDisease(s,c,p,stages)

Disease: black_mold, Symptom: ['lesions'], SymptomColour: ['black'], AffectsPart: ['leaf']
